In [1]:
#importing the Pytorch Libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
#Model Architecture
class Net(nn.Module):
    def __init__(self):
        """ This function instantiates all the model layers """
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)               # Input: 28*28*1   OUtput:26*26*16 | RF:3*3
        self.normalisation1=nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout(0.15)
        
        self.conv2 = nn.Conv2d(16, 16, 3)            # Input: 26*26*1   OUtput:24*24*16 | RF:5*5
        self.normalisation2=nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout(0.15)
        
        self.conv3 = nn.Conv2d(16, 16, 3)          # Input: 24*24*1   OUtput:22*22*16 | RF:7*7
        self.normalisation3=nn.BatchNorm2d(16)
        
        self.pool = nn.MaxPool2d(2, 2)             # Input: 22*22*16   OUtput:11*11*16 | RF:14*14
        
        self.conv4 = nn.Conv2d(16, 16, 3)         # Input: 11*11*16   OUtput:9*9*16 | RF:16*16
        self.normalisation4=nn.BatchNorm2d(16)
        self.dropout3 = nn.Dropout(0.15)
        
        self.conv5 = nn.Conv2d(16,16, 3)          # Input: 9*9*16   OUtput:7*7*16 | RF:18*18
        self.normalisation5=nn.BatchNorm2d(16)
        self.dropout4 = nn.Dropout(0.2)
        
        self.conv6 = nn.Conv2d(16, 10, 3)         # Input: 7*7*1   OUtput:5*5*16 | RF:20*20
        self.gap = nn.AdaptiveAvgPool2d(1)
        

    def forward(self, x):
        x = self.normalisation1(F.relu(self.conv1(x)))
        x = self.dropout2(self.normalisation2(F.relu(self.conv2(x))))
        x = self.pool(self.normalisation3(F.relu(self.conv3(x))))
        x = self.dropout3(self.normalisation4(F.relu(self.conv4(x))))
        x = self.dropout4(self.normalisation5(F.relu(self.conv5(x))))
        x = F.relu(self.conv6(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [8]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 16, 24, 24]           2,320
       BatchNorm2d-4           [-1, 16, 24, 24]              32
           Dropout-5           [-1, 16, 24, 24]               0
            Conv2d-6           [-1, 16, 22, 22]           2,320
       BatchNorm2d-7           [-1, 16, 22, 22]              32
         MaxPool2d-8           [-1, 16, 11, 11]               0
            Conv2d-9             [-1, 16, 9, 9]           2,320
      BatchNorm2d-10             [-1, 16, 9, 9]              32
          Dropout-11             [-1, 16, 9, 9]               0
           Conv2d-12             [-1, 16, 7, 7]           2,320
      BatchNorm2d-13             [-1, 16, 7, 7]              32
          Dropout-14             [-1, 1

You should consider upgrading via the 'c:\users\nei3kor\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.
c:\users\nei3kor\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [9]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [10]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0664, Accuracy: 9815/10000 (98.15%)



  0%|                                                                                          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0372, Accuracy: 9883/10000 (98.83%)



  0%|                                                                                          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0233, Accuracy: 9924/10000 (99.24%)



  0%|                                                                                          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0222, Accuracy: 9932/10000 (99.32%)



loss=0.12844759225845337 batch_id=937: 100%|█████████████████████████████████████████| 938/938 [00:21<00:00, 43.19it/s]



Test set: Average loss: 0.0191, Accuracy: 9940/10000 (99.40%)



 #  With dropout 0.15 and 0.2 got 99.4% accuracy
 # In the previous run with dropout 0.25 got 99.39% accuracy

In [221]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0576, Accuracy: 9827/10000 (98.27%)



  0%|                                                                                          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0358, Accuracy: 9892/10000 (98.92%)



  0%|                                                                                          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0215, Accuracy: 9934/10000 (99.34%)



  0%|                                                                                          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0242, Accuracy: 9923/10000 (99.23%)



loss=0.04795098304748535 batch_id=937: 100%|█████████████████████████████████████████| 938/938 [00:25<00:00, 36.54it/s]



Test set: Average loss: 0.0207, Accuracy: 9939/10000 (99.39%)

